# Generative Agents in LangChain by Claude haiku

This notebook implements a generative agent based on the paper [Generative Agents: Interactive Simulacra of Human Behavior](https://arxiv.org/abs/2304.03442) by Park, et. al.

In it, we leverage a time-weighted Memory object backed by a LangChain Retriever.

In [30]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import logging

logging.basicConfig(level=logging.ERROR)

In [2]:
from datetime import datetime, timedelta
from typing import List

from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from langchain.chat_models import ChatAnthropic
from langchain_anthropic import ChatAnthropic
from termcolor import colored
from env import API_KEY, VOYAGE_API_KEY
import os
import voyageai

In [3]:
USER_NAME = "Person A"  # The name you want to use when interviewing the agent.
# LLM = ChatOpenAI(max_tokens=1500)  # Can be any LLM you want.
os.environ["ANTHROPIC_API_KEY"] = API_KEY
os.environ["VOYAGE_API_KEY"] = VOYAGE_API_KEY
LLM = ChatAnthropic(
    model="claude-3-haiku-20240307",
    max_tokens=1500
)

### Generative Agent Memory Components

This tutorial highlights the memory of generative agents and its impact on their behavior. The memory varies from standard LangChain Chat memory in two aspects:

1. **Memory Formation**

   Generative Agents have extended memories, stored in a single stream:
      1. Observations - from dialogues or interactions with the virtual world, about self or others
      2. Reflections - resurfaced and summarized core memories


2. **Memory Recall**

   Memories are retrieved using a weighted sum of salience, recency, and importance.

You can review the definitions of the `GenerativeAgent` and `GenerativeAgentMemory` in the [reference documentation]("https://api.python.langchain.com/en/latest/modules/experimental.html") for the following imports, focusing on `add_memory` and `summarize_related_memories` methods.

In [4]:
from langchain_experimental.generative_agents import (
    GenerativeAgent,
    GenerativeAgentMemory,
)


from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

# lc_embed_model = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-mpnet-base-v2"
# )
# embed_model = LangchainEmbedding(lc_embed_model)

In [18]:
embed_model = HuggingFaceEmbeddings()
out = embed_model.embed_query("hello")
embed_model_size = len(out)

## Memory Lifecycle

Summarizing the key methods in the above: `add_memory` and `summarize_related_memories`.

When an agent makes an observation, it stores the memory:
    
1. Language model scores the memory's importance (1 for mundane, 10 for poignant)
2. Observation and importance are stored within a document by TimeWeightedVectorStoreRetriever, with a `last_accessed_time`.

When an agent responds to an observation:

1. Generates query(s) for retriever, which fetches documents based on salience, recency, and importance.
2. Summarizes the retrieved information
3. Updates the `last_accessed_time` for the used documents.


## Create a Generative Character



Now that we've walked through the definition, we will create two characters named "Tommie" and "Eve".

In [11]:
import math

import faiss

from transformers import AutoTokenizer, AutoModel
import torch

# from sentence_transformers import SentenceTransformer


In [19]:

# class HuggingFaceEmbeddings:
#     def __init__(self, model_name="all-MiniLM-L6-v2"):
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name)
#         self.model = AutoModel.from_pretrained(model_name)

#     def embed_query(self, text):
#         inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True)
#         with torch.no_grad():
#             outputs = self.model(**inputs)
#         return outputs
#         # return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# embed_model = HuggingFaceEmbeddings()


def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    # This will differ depending on a few things:
    # - the distance / similarity metric used by the VectorStore
    # - the scale of your embeddings (OpenAI's are unit norm. Many others are not!)
    # This function converts the euclidean norm of normalized embeddings
    # (0 is most similar, sqrt(2) most dissimilar)
    # to a similarity function (0 to 1)
    return 1.0 - score / math.sqrt(2)


def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    # embeddings_model = OpenAIEmbeddings()
    embeddings_model = embed_model
    # Initialize the vectorstore as empty
    # embedding_size = 1536
    embedding_size = embed_model_size

    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,
        index,
        InMemoryDocstore({}),
        {},
        relevance_score_fn=relevance_score_fn,
    )
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore, other_score_keys=["importance"], k=15
    )

In [20]:
tommies_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
)

tommie = GenerativeAgent(
    name="Tommie",
    age=25,
    traits="anxious, likes design, talkative",  # You can add more persistent traits here
    status="looking for a job",  # When connected to a virtual world, we can have the characters update their status
    memory_retriever=create_new_memory_retriever(),
    llm=LLM,
    memory=tommies_memory,
)

In [21]:
# The current "Summary" of a character can't be made because the agent hasn't made
# any observations yet.
print(tommie.get_summary())

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Based on the provided statements, Tommie can be summarized as follows:

Tommie is a diligent and hardworking individual. They take their responsibilities seriously and strive to complete tasks efficiently. Tommie is also a reliable and dependable person, as they can be counted on to fulfill their commitments. Additionally, Tommie is described as thoughtful and considerate, taking the time to understand and address the needs of others. Overall, Tommie appears to be a dedicated and conscientious person.


In [22]:
# We can add memories directly to the memory object
tommie_observations = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]
for observation in tommie_observations:
    tommie.memory.add_memory(observation)

In [23]:
# Now that Tommie has 'memories', their self-summary is more descriptive, though still rudimentary.
# We will see how this summary updates after more observations to create a more rich description.
print(tommie.get_summary(force_refresh=True))

/usr/local/anaconda3/envs/v2l/lib/python3.11/site-packages/langchain_core/vectorstores.py:379: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Tommie tries to get some rest.', metadata={'importance': 0.03, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 13, 29, 768699), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 29, 768699), 'buffer_idx': 6}), 0.44842040032156383), (Document(page_content='Tommie sees the new home', metadata={'importance': 0.075, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 13, 25, 834877), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 25, 834877), 'buffer_idx': 2}), 0.4454932173987852), (Document(page_content='Tommie is hungry', metadata={'importance': 0.03, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 13, 29, 175342), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 29, 175342), 'buffer_idx': 5}), 0.43128058358448706), (Document(page_content='Tommie remembers his dog, Bruno, from when he was

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Based on the provided statements, the core characteristics of Tommie can be summarized as follows:

Tommie is a practical and observant individual. He takes note of his surroundings, such as the new home he sees and the new neighbors with a cat. Tommie also seems attentive to his own physical needs, as he tries to get some rest and acknowledges feeling hungry and tired from the long drive. Additionally, Tommie appears to have a nostalgic side, as he remembers his childhood dog, Bruno. Overall, Tommie comes across as a grounded and self-aware person who is responsive to his environment and personal experiences.


## Pre-Interview with Character

Before sending our character on their way, let's ask them a few questions.

In [24]:
def interview_agent(agent: GenerativeAgent, message: str) -> str:
    """Help the notebook user interact with the agent."""
    new_message = f"{USER_NAME} says {message}"
    return agent.generate_dialogue_response(new_message)[1]

In [25]:
interview_agent(tommie, "What do you like to do?")

/usr/local/anaconda3/envs/v2l/lib/python3.11/site-packages/langchain_core/vectorstores.py:379: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Tommie is hungry', metadata={'importance': 0.03, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 13, 31, 684359), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 29, 175342), 'buffer_idx': 5}), 0.21873128699889532), (Document(page_content='Tommie sees the new home', metadata={'importance': 0.075, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 13, 31, 684359), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 25, 834877), 'buffer_idx': 2}), 0.191435554910391), (Document(page_content='Tommie tries to get some rest.', metadata={'importance': 0.03, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 13, 31, 684359), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 29, 768699), 'buffer_idx': 6}), 0.1564643759458707), (Document(page_content='Tommie remembers his dog, Bruno, from when he was a

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

'Tommie said "Well, as you can probably tell, I\'m a bit of a busy body and I like to keep my eyes and ears open. I tend to get anxious at times, but design and creative projects really help me unwind. And, you know, I just love a good conversation - I could talk the ear off anyone who\'s willing to listen!"'

In [27]:
# interview_agent(tommie, "What are you looking forward to doing today?")
interview_agent(tommie, "今日は何をする予定ですか？")

/usr/local/anaconda3/envs/v2l/lib/python3.11/site-packages/langchain_core/vectorstores.py:379: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Tommie sees the new home', metadata={'importance': 0.075, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 14, 36, 746671), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 25, 834877), 'buffer_idx': 2}), 0.23288689612715818), (Document(page_content='Tommie is hungry', metadata={'importance': 0.03, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 14, 36, 746671), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 29, 175342), 'buffer_idx': 5}), 0.22659184283357103), (Document(page_content='Tommie observed Person A says What are you looking forward to doing today? and said "Well, to be honest, I\'m still getting settled in and trying to figure out my next steps. This move has been a bit of a whirlwind, you know? But I\'m looking forward to exploring the neighborhood, maybe checking out some of the

'Tommie said "Well, to be honest, I\'m still getting settled in and trying to figure out my next steps. This move has been a bit of a whirlwind, you know? But I\'m looking forward to exploring the neighborhood, maybe checking out some of the local shops and restaurants. And of course, I\'m hoping to find a great job opportunity that really aligns with my interests and skills. Design and creative work is such a passion of mine, so I\'m keeping my eyes peeled for any openings in that field. In the meantime, I\'m just trying to take it one day at a time, get some rest, and maybe do a little networking to see what possibilities might be out there. What about you? Do you have any recommendations for good spots to check out around here?"'

In [13]:
interview_agent(tommie, "What are you most worried about today?")

'Tommie said "Honestly, I\'m feeling pretty anxious about finding a job. It\'s been a bit of a struggle lately, but I\'m trying to stay positive and keep searching. How about you, Person A? What worries you?"'

## Step through the day's observations.

In [28]:
# Let's have Tommie start going through a day in the life.
observations = [
    "Tommie wakes up to the sound of a noisy construction site outside his window.",
    "Tommie gets out of bed and heads to the kitchen to make himself some coffee.",
    "Tommie realizes he forgot to buy coffee filters and starts rummaging through his moving boxes to find some.",
    "Tommie finally finds the filters and makes himself a cup of coffee.",
    "The coffee tastes bitter, and Tommie regrets not buying a better brand.",
    "Tommie checks his email and sees that he has no job offers yet.",
    "Tommie spends some time updating his resume and cover letter.",
    "Tommie heads out to explore the city and look for job openings.",
    "Tommie sees a sign for a job fair and decides to attend.",
    "The line to get in is long, and Tommie has to wait for an hour.",
    "Tommie meets several potential employers at the job fair but doesn't receive any offers.",
    "Tommie leaves the job fair feeling disappointed.",
    "Tommie stops by a local diner to grab some lunch.",
    "The service is slow, and Tommie has to wait for 30 minutes to get his food.",
    "Tommie overhears a conversation at the next table about a job opening.",
    "Tommie asks the diners about the job opening and gets some information about the company.",
    "Tommie decides to apply for the job and sends his resume and cover letter.",
    "Tommie continues his search for job openings and drops off his resume at several local businesses.",
    "Tommie takes a break from his job search to go for a walk in a nearby park.",
    "A dog approaches and licks Tommie's feet, and he pets it for a few minutes.",
    "Tommie sees a group of people playing frisbee and decides to join in.",
    "Tommie has fun playing frisbee but gets hit in the face with the frisbee and hurts his nose.",
    "Tommie goes back to his apartment to rest for a bit.",
    "A raccoon tore open the trash bag outside his apartment, and the garbage is all over the floor.",
    "Tommie starts to feel frustrated with his job search.",
    "Tommie calls his best friend to vent about his struggles.",
    "Tommie's friend offers some words of encouragement and tells him to keep trying.",
    "Tommie feels slightly better after talking to his friend.",
]

In [29]:
# Let's send Tommie on their way. We'll check in on their summary every few observations to watch it evolve
for i, observation in enumerate(observations):
    _, reaction = tommie.generate_reaction(observation)
    print(colored(observation, "green"), reaction)
    if ((i + 1) % 20) == 0:
        print("*" * 40)
        print(
            colored(
                f"After {i+1} observations, Tommie's summary is:\n{tommie.get_summary(force_refresh=True)}",
                "blue",
            )
        )
        print("*" * 40)

/usr/local/anaconda3/envs/v2l/lib/python3.11/site-packages/langchain_core/vectorstores.py:379: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Tommie sees the new home', metadata={'importance': 0.075, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 15, 28, 960782), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 25, 834877), 'buffer_idx': 2}), 0.2230643202006033), (Document(page_content='Tommie is hungry', metadata={'importance': 0.03, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 15, 28, 960782), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 29, 175342), 'buffer_idx': 5}), 0.1219622914052767), (Document(page_content='Tommie tries to get some rest.', metadata={'importance': 0.03, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 15, 28, 960782), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 29, 768699), 'buffer_idx': 6}), 0.11096904461448309), (Document(page_content='Tommie observed Person A says What do you like to 

Tommie wakes up to the sound of a noisy construction site outside his window. Tommie said "Ugh, this noisy construction is going to make it hard for me to get any rest. Guess I'd better get up and find something to distract myself with until it dies down."


/usr/local/anaconda3/envs/v2l/lib/python3.11/site-packages/langchain_core/vectorstores.py:379: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Tommie is hungry', metadata={'importance': 0.03, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 16, 24, 210202), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 29, 175342), 'buffer_idx': 5}), 0.20030653929126696), (Document(page_content='Tommie sees the new home', metadata={'importance': 0.075, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 16, 24, 210202), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 25, 834877), 'buffer_idx': 2}), 0.19333123586271395), (Document(page_content='Tommie observed Tommie wakes up to the sound of a noisy construction site outside his window. and reacted by SAY: "Ugh, this noisy construction is going to make it hard for me to get any rest. Guess I\'d better get up and find something to distract myself with until it dies down."', metadata={'importance': 0.03

Tommie gets out of bed and heads to the kitchen to make himself some coffee. Tommie said "Ah, a fresh cup of coffee is just what I need to start the day. This should help me feel more alert and ready to tackle whatever comes my way."


/usr/local/anaconda3/envs/v2l/lib/python3.11/site-packages/langchain_core/vectorstores.py:379: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Tommie sees the new home', metadata={'importance': 0.075, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 16, 30, 285518), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 25, 834877), 'buffer_idx': 2}), 0.33566923547891714), (Document(page_content='Tommie observed Tommie wakes up to the sound of a noisy construction site outside his window. and reacted by SAY: "Ugh, this noisy construction is going to make it hard for me to get any rest. Guess I\'d better get up and find something to distract myself with until it dies down."', metadata={'importance': 0.03, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 16, 30, 285518), 'created_at': datetime.datetime(2024, 7, 4, 13, 16, 27, 865182), 'buffer_idx': 10}), 0.2929636461973142), (Document(page_content='Tommie is hungry', metadata={'importance': 0.03

...

Tommie checks his email and sees that he has no job offers yet. Tommie said "Hmm, still no job offers yet. Guess I'll have to keep searching and networking. I'm sure the right opportunity will come along, I just need to stay positive and keep putting myself out there."


/usr/local/anaconda3/envs/v2l/lib/python3.11/site-packages/langchain_core/vectorstores.py:379: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Tommie sees the new home', metadata={'importance': 0.075, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 16, 59, 645370), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 25, 834877), 'buffer_idx': 2}), 0.0751958111228006), (Document(page_content='Tommie is hungry', metadata={'importance': 0.03, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 16, 59, 645370), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 29, 175342), 'buffer_idx': 5}), 0.07244218892218979), (Document(page_content='Tommie observed Person A says What are you looking forward to doing today? and said "Well, to be honest, I\'m still getting settled in and trying to figure out my next steps. This move has been a bit of a whirlwind, you know? But I\'m looking forward to exploring the neighborhood, maybe checking out some of the 

Tommie spends some time updating his resume and cover letter. Tommie said "Updating my resume and cover letter is an important first step in my job search. I want to make sure I'm presenting myself in the best possible light to potential employers. With a little luck, this hard work will pay off and help me find the right fit for my skills and experience."


/usr/local/anaconda3/envs/v2l/lib/python3.11/site-packages/langchain_core/vectorstores.py:379: UserWarning: Relevance scores must be between 0 and 1, got [(Document(page_content='Tommie sees the new home', metadata={'importance': 0.075, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 17, 6, 928442), 'created_at': datetime.datetime(2024, 7, 4, 13, 13, 25, 834877), 'buffer_idx': 2}), 0.2337845397067433), (Document(page_content='Tommie observed Tommie spends some time updating his resume and cover letter. and reacted by SAY: "Updating my resume and cover letter is an important first step in my job search. I want to make sure I\'m presenting myself in the best possible light to potential employers. With a little luck, this hard work will pay off and help me find the right fit for my skills and experience."', metadata={'importance': 0.045, 'last_accessed_at': datetime.datetime(2024, 7, 4, 13, 17, 10, 664122), 'created_at': datetime.datetime(2024, 7, 4, 13, 17, 10, 664122), 'buffer_idx

KeyboardInterrupt: 

## Interview after the day

In [16]:
interview_agent(tommie, "Tell me about how your day has been going")

'Tommie said "It\'s been a bit of a rollercoaster, to be honest. I\'ve had some setbacks in my job search, but I also had some good moments today, like sending out a few resumes and meeting some potential employers at a job fair. How about you?"'

In [17]:
interview_agent(tommie, "How do you feel about coffee?")

'Tommie said "I really enjoy coffee, but sometimes I regret not buying a better brand. How about you?"'

In [31]:
interview_agent(tommie, "Tell me about your childhood dog!")

'Tommie said "Ah, my childhood dog Bruno - those were such happy times! He was this big, shaggy mutt that I absolutely adored. I have so many fond memories of playing fetch with him in the backyard, going for long walks around the neighborhood, and just snuggling up with him on the couch. Bruno was such a loyal and loving companion. He really was the best friend a kid could ask for. I miss his goofy personality and wagging tail so much. Those memories of growing up with him still bring a smile to my face, even all these years later. Do you have any pets or special animals from your own childhood that you remember fondly?"'

## Adding Multiple Characters

Let's add a second character to have a conversation with Tommie. Feel free to configure different traits.

In [32]:
eves_memory = GenerativeAgentMemory(
    llm=LLM,
    memory_retriever=create_new_memory_retriever(),
    verbose=False,
    reflection_threshold=5,
)


eve = GenerativeAgent(
    name="Eve",
    age=34,
    traits="curious, helpful",  # You can add more persistent traits here
    status="N/A",  # When connected to a virtual world, we can have the characters update their status
    llm=LLM,
    daily_summaries=[
        (
            "Eve started her new job as a career counselor last week and received her first assignment, a client named Tommie."
        )
    ],
    memory=eves_memory,
    verbose=False,
)

In [33]:
yesterday = (datetime.now() - timedelta(days=1)).strftime("%A %B %d")
eve_observations = [
    "Eve wakes up and hear's the alarm",
    "Eve eats a boal of porridge",
    "Eve helps a coworker on a task",
    "Eve plays tennis with her friend Xu before going to work",
    "Eve overhears her colleague say something about Tommie being hard to work with",
]
for observation in eve_observations:
    eve.memory.add_memory(observation)

In [34]:
print(eve.get_summary())

Name: Eve (age: 34)
Innate traits: curious, helpful
Based on the provided statements, here is a summary of Eve's core characteristics:

Eve appears to be a diligent and hardworking individual. She starts her day early, waking up to the sound of an alarm, and has a healthy breakfast of porridge. She is also observant, as she overhears a conversation about a coworker's behavior. Additionally, Eve seems to be helpful and collaborative, as she assists a coworker with a task. Finally, Eve maintains a balanced lifestyle, taking time to engage in recreational activities like playing tennis with a friend before going to work.

Overall, the statements suggest that Eve is a responsible, attentive, and considerate person who strives to maintain a healthy work-life balance.


## Pre-conversation interviews


Let's "Interview" Eve before she speaks with Tommie.

In [50]:
interview_agent(eve, "How are you feeling about today?")

'Eve said "I\'m feeling pretty good, thanks for asking! Just trying to stay productive and make the most of the day. How about you?"'

In [51]:
interview_agent(eve, "What do you know about Tommie?")

'Eve said "I don\'t know much about Tommie, but I heard someone mention that they find them difficult to work with. Have you had any experiences working with Tommie?"'

In [35]:
interview_agent(
    eve,
    "Tommie is looking to find a job. What are are some things you'd like to ask him?",
)

'Eve said "I\'m sorry, I don\'t actually have any direct relationship or connection with Tommie. The details provided focused on Tommie\'s job search process, but didn\'t mention any involvement from me. Without more context about my relationship to Tommie, I don\'t have enough information to provide specific advice or questions to ask him. I\'d be happy to have a thoughtful discussion if you can share more details about the nature of my involvement, if any, with Tommie\'s situation. Otherwise, I\'m afraid I can\'t offer any personal insights or recommendations in this case. Please let me know if you have any other questions I could assist with."'

In [53]:
interview_agent(
    eve,
    "You'll have to ask him. He may be a bit anxious, so I'd appreciate it if you keep the conversation going and ask as many questions as possible.",
)

'Eve said "Sure, I can keep the conversation going and ask plenty of questions. I want to make sure Tommie feels comfortable and supported. Thanks for letting me know."'

## Dialogue between Generative Agents

Generative agents are much more complex when they interact with a virtual environment or with each other. Below, we run a simple conversation between Tommie and Eve.

In [36]:
def run_conversation(agents: List[GenerativeAgent], initial_observation: str) -> None:
    """Runs a conversation between agents."""
    _, observation = agents[1].generate_reaction(initial_observation)
    print(observation)
    turns = 0
    while True:
        break_dialogue = False
        for agent in agents:
            stay_in_dialogue, observation = agent.generate_dialogue_response(
                observation
            )
            print(observation)
            # observation = f"{agent.name} said {reaction}"
            if not stay_in_dialogue:
                break_dialogue = True
        if break_dialogue:
            break
        turns += 1

In [37]:
agents = [tommie, eve]
run_conversation(
    agents,
    "Tommie said: Hi, Eve. Thanks for agreeing to meet with me today. I have a bunch of questions and am not sure where to start. Maybe you could first share about your experience?",
)

Eve said "Of course, Tommie. I'm happy to share my perspective and answer any questions you might have. As I mentioned earlier, I don't have a direct personal relationship with you, but I'm glad to provide any insights I can based on the information available to me. Where would you like me to start?"
Tommie said "That's great, Eve. I appreciate you taking the time to share your thoughts and insights. As I mentioned, I'm in the midst of a job search, so I'd be really interested to hear your perspective on that. Do you have any advice or suggestions for me as I navigate the process of finding the right job? I'm feeling a bit anxious about it, but I'm trying to stay positive and keep putting myself out there."


KeyboardInterrupt: 

## Let's interview our agents after their conversation

Since the generative agents retain their memories from the day, we can ask them about their plans, conversations, and other memoreis.

In [56]:
# We can see a current "Summary" of a character based on their own perception of self
# has changed
print(tommie.get_summary(force_refresh=True))

Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is determined and hopeful in his job search, but can also feel discouraged and frustrated at times. He has a strong connection to his childhood dog, Bruno. Tommie seeks support from his friends when feeling overwhelmed and is grateful for their help. He also enjoys exploring his new city.


In [57]:
print(eve.get_summary(force_refresh=True))

Name: Eve (age: 34)
Innate traits: curious, helpful
Eve is a helpful and friendly person who enjoys playing sports and staying productive. She is attentive and responsive to others' needs, actively listening and asking questions to understand their perspectives. Eve has experience in event planning and communication, and is willing to share her knowledge and expertise with others. She values teamwork and collaboration, and strives to create a comfortable and supportive environment for everyone.


In [58]:
interview_agent(tommie, "How was your conversation with Eve?")

'Tommie said "It was really helpful actually. Eve shared some great tips on managing events and handling unexpected issues. I feel like I learned a lot from her experience."'

In [59]:
interview_agent(eve, "How was your conversation with Tommie?")

'Eve said "It was great, thanks for asking. Tommie was very receptive and had some great questions about event planning. How about you, have you had any interactions with Tommie?"'

In [60]:
interview_agent(eve, "What do you wish you would have said to Tommie?")

'Eve said "It was great meeting with you, Tommie. If you have any more questions or need any help in the future, don\'t hesitate to reach out to me. Have a great day!"'